In [ ]:
import pandas as pd
import numpy as np
import geopy.distance
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
data = pd.read_csv('/home/lakshikka/Desktop/Semester 07/ML/project/train.csv', 
                   parse_dates = ['pickup_time', 'drop_time']).drop(columns = 'tripid')
df_test = pd.read_csv('/home/lakshikka/Desktop/Semester 07/ML/project/test.csv', 
                   parse_dates = ['pickup_time', 'drop_time']).drop(columns = 'tripid')

# Remove na
data = data.dropna()
df_test = df_test.dropna()
data.head()

data['label'] = data['label'].map( {'incorrect':0, 'correct':1})

In [ ]:
#data.describe()

In [ ]:
#data.info()

In [ ]:
#creating new column for meter waiting rate
data['meter_waiting_rate'] = data['meter_waiting_fare']/data['meter_waiting']
data['meter_waiting_rate'] = data['meter_waiting_rate'].fillna(0.0)

df_test['meter_waiting_rate'] = df_test['meter_waiting_fare']/df_test['meter_waiting']
df_test['meter_waiting_rate'] = df_test['meter_waiting_rate'].fillna(0.0)


In [ ]:
#Creating new columns for train and test data from pickup and drop time

data['p_year'] = data.pickup_time.dt.year
data['p_month'] = data.pickup_time.dt.month
data['p_day'] = data.pickup_time.dt.day
data['p_hour'] = data.pickup_time.dt.hour
data['p_min'] = data.pickup_time.dt.minute
data['p_weekday'] = data.pickup_time.dt.weekday_name


data['d_year'] = data.drop_time.dt.year
data['d_month'] = data.drop_time.dt.month
data['d_day'] = data.drop_time.dt.day
data['d_hour'] = data.drop_time.dt.hour
data['d_min'] = data.drop_time.dt.minute
data['d_weekday'] = data.drop_time.dt.weekday_name

df_test['p_year'] = df_test.pickup_time.dt.year
df_test['p_month'] = df_test.pickup_time.dt.month
df_test['p_day'] = df_test.pickup_time.dt.day
df_test['p_hour'] = df_test.pickup_time.dt.hour
df_test['p_min'] = df_test.pickup_time.dt.minute
df_test['p_weekday'] = df_test.pickup_time.dt.weekday_name

df_test['d_year'] = df_test.drop_time.dt.year
df_test['d_month'] = df_test.drop_time.dt.month
df_test['d_day'] = df_test.drop_time.dt.day
df_test['d_hour'] = df_test.drop_time.dt.hour
df_test['d_min'] = df_test.drop_time.dt.minute
df_test['d_weekday'] = df_test.drop_time.dt.weekday_name

In [ ]:
#Distance calculation of the trip
def distance(row):
    return geopy.distance.distance((row.pick_lat, row.pick_lon),(row.drop_lat, row.drop_lon)).mi

data['distance']= data.apply(lambda r: distance(r), axis=1)
df_test['distance']= data.apply(lambda r: distance(r), axis=1)



In [ ]:
#Encoding categorical values for weekdays
lb_make = LabelEncoder()

data['p_weekday'] = lb_make.fit_transform(data['p_weekday'])
data['d_weekday'] = lb_make.fit_transform(data['d_weekday'])

df_test['p_weekday'] = lb_make.fit_transform(df_test['p_weekday'])
df_test['d_weekday'] = lb_make.fit_transform(df_test['d_weekday'])


In [ ]:
from sklearn.model_selection import train_test_split

df_train = data[['additional_fare','duration', 'meter_waiting_rate', 'meter_waiting_till_pickup', 'pick_lat', 'pick_lon','drop_lat','drop_lon', 'fare', 'p_year' , 'p_month', 'p_day','p_hour','p_min','d_day','d_hour','d_min','distance', 'p_weekday','d_weekday' ]]
df_label = data[['label']]

#X_train_1, X_test_1, Y_train_1, Y_test_1 = train_test_split(df_train, df_train, test_size=0.2)
df_test_sample = df_test[['additional_fare','duration', 'meter_waiting_rate', 'meter_waiting_till_pickup', 'pick_lat', 'pick_lon','drop_lat','drop_lon', 'fare', 'p_year' , 'p_month', 'p_day','p_hour','p_min','d_day','d_hour','d_min','distance' , 'p_weekday','d_weekday']]

# Split our data
train, test, train_labels, test_labels = train_test_split(df_train,
                                                          df_label,
                                                          test_size=0.33,
                                                          random_state=42)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import f1_score

clf_xgb = XGBClassifier(objective = 'binary:logistic')
param_dist = { "n_estimators" :[100,200,500,1000],
    "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] }

clf = RandomizedSearchCV(clf_xgb, param_distributions = param_dist, n_iter = 25, scoring = 'f1', error_score = 0, verbose = 3, n_jobs = -1)
clf = clf.fit(train, train_labels)

# Results from Random Search
print("\n========================================================")
print(" Results from Random Search " )
print("========================================================")
print("\n The best estimator across ALL searched params:\n",
          clf.best_estimator_)
print("\n The best score across ALL searched params:\n",
          clf.best_score_)
print("\n The best parameters across ALL searched params:\n",
          clf.best_params_)
print("\n ========================================================")


In [ ]:
# import xgboost as xgb
# xg_reg = xgb.XGBClassifier(objective ='binary:logistic',n_estimators = 1000,  learning_rate = 0.15, colsample_bytree=0.7,
#                 max_depth = 6, gamma = 0.0, min_child_weight= 5)


# xg_reg.fit(train, train_labels)
# preds = xg_reg.predict(test)
# print(f1_score(test_labels, preds, average="macro"))

In [ ]:
import xgboost as xgb
xg_reg = xgb.XGBClassifier(objective ='binary:logistic',  learning_rate = 0.1, colsample_bytree=0.7,
                max_depth = 12, gamma = 0.4, min_child_weight= 5)


xg_reg.fit(df_train,df_label)
Y_pred = xg_reg.predict(df_test_sample)
#print(f1_score(test_labels, Y_pred, average="macro"))

In [ ]:
# submission_format = pd.read_csv('/home/lakshikka/Desktop/Semester 07/ML/project/sample_submission.csv', index_col='tripid')
# my_submission = pd.DataFrame(data=Y_pred,
#                              columns=submission_format.columns,
#                              index=submission_format.index)
# my_submission.to_csv('/home/lakshikka/Desktop/submission_Distance_04.csv')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

(pd.Series(xg_reg.feature_importances_, index=train.columns)
   .nlargest(30)
   .plot(kind='barh'))